# CS486 - Artificial Intelligence
## Lesson 14 - Markov Decision Processes

*Expectimax* is a way to search a tree for the best action when outcomes are ucertain. In practice, however, we can rarely  search to the root of an expectimax tree. 

`Markov Decision Processes` (MDPs) are a way of formulating problems such that we can use an *expectimax* approach to establish a **policy** for selecting optimal actions in states without having to perform a search every time.  

In [ ]:
import helpers
from aima.text import *
from aima.notebook import psource

## A Draw HiLo Policy

Last time we looked at using `expectimax` to decide the best action to take when playing *Draw HiLo*. Our implementation was limited our game to 5 draws since, in theory, might never end otherwise. 

Why exactly did we need `expectimax` to search the tree? Do future our past draws effect which action is optimal for the current draw? It doesn't, just like the last result of a coin flip has no bearing on the next. So an `expectimax` search of depth 1 will yield the same result as any deeper search. 

Running an `expectimax` of depth 1 in every state will produce the optimal action in each state. Together, the set of optimal actions for all states in an MDP is called the **policy**. Here is the policy for *Draw HiLo*:

| Draw | Action | Exp. Return | 
| - | - | - |
| 2 | H | 1.008 |
| 3 | H | 1.000 |
| 4 | L | 0.975 |
| 5 | L | 0.933 |
| 6 | H | 0.875 |
| 7 | - | 0.900 |
| 8 | L | 0.833 |
| 9 | H | 1.000 |
| 10 | H | 1.000 |
| J | L | 0.833 |
| Q | L | 1.000 |

## Value Iteration